In [1]:
#loading libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
#import the dataset
frame=pd.read_csv('train.csv')
pd.set_option('display.max.columns',100)
pd.set_option('display.max.rows',90)

In [2]:
frame.drop(['Id'],axis=1,inplace=True)

In [3]:
frame.fillna(frame.mean(),inplace=True)

In [4]:
frame.drop(['PoolQC','MiscFeature','Fence','Alley',"OverallQual","YearBuilt","TotalBsmtSF","GarageCars","GrLivArea"],axis=1,inplace=True)

In [5]:
#get name columns with numeric and object types \n",
def get_object_columns(dataframe):
    object_columns=[columns_name for columns_name in dataframe.columns if dataframe[columns_name].dtypes == object]
    return object_columns
def get_numric_columns(dataframe):
    numeric=[columns_name for columns_name in dataframe.columns if dataframe[columns_name].dtypes != object]
    return numeric
print(get_object_columns(frame),'\n'*3)
print(get_numric_columns(frame))

['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition'] 



['MSSubClass', 'LotFrontage', 'LotArea', 'OverallCond', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']


In [6]:
def find_KNN_for_fillna(dataframe):
    def dist(first_obj, second_obj):  # растояние(евклидова метрика) между объектами
        first_obj = list(first_obj)
        second_obj = list(second_obj)
        dist = 0
        for i in range(len(first_obj)):
            #             print(first_obj[i],'first')
            #             print(second_obj[i],'second')
            dist += (float(first_obj[i]) - float(second_obj[i])) ** 2
        dist = dist ** 0.5
        #         print(first_obj)
        #         print(np.array(first_obj,dtype=int))
        #         print(second_obj)
        #        dist = (sum((np.array(first_obj) - second_obj) ** 2)) ** 0.5
        return dist

    def new_dataframe(name_column, dataframe):  # создает новый датафрейм, где не содержится NaN в целевом столбце
        dataset = pd.DataFrame([dataframe.iloc[i] for i, res in enumerate(dataframe[name_column]) if res is not np.nan])
        dataset.dropna(inplace=True)
        return dataset

    def preparing_data(global_data, dataframe, column_number,
                       row_number):  # подготовка всей таблицы к KNN и получение тестового объекта
        test_object = pd.DataFrame(global_data.iloc[row_number])
        all_headers = dataframe.columns.values
        null_mass = test_object.isnull()
        null_mass = [i[0] for i in np.array(null_mass)]
        mult = null_mass * np.array(all_headers)
        columns_to_del = [i for i in mult if i != ""]
        column_class = dataframe[column_number]
        data = pd.DataFrame(dataframe.drop(columns_to_del + [column_number], axis=1))
        test_object = test_object.drop(columns_to_del)
        test_object = [i[0] for i in np.array(test_object)]
        total = []
        if len(data) == len(column_class):
            for i in range(len(data)):
                total.append([list(data.iloc[i]), np.array(column_class)[i]])
        else:
            print("OMG")

        return total, test_object

    def encoding(dataframe):
        unique_cat = []
        obj_col = get_object_columns(dataframe)
        for i in obj_col:  # находим уникальные значения в каждом столбце
            unique_cat += dataframe[i].unique().tolist()
        while np.nan in unique_cat:
            ind = unique_cat.index(np.nan)
            unique_cat.pop(ind)
        encoder = LabelEncoder()  # кодируем полученные значения и заменяим их
        encoder.fit(unique_cat)
        for column in obj_col:
            for row in range(len(dataframe)):
                if dataframe[column][row] is not np.nan:
                    dataframe[column][row] = encoder.transform([dataframe[column][row]])[0]
        return dataframe, encoder

    def nn(train_data, test_point):  # функция ищет ближайшего соседа, а затем получает иго искомое значение
        test_dist = [[dist(test_point, data[0]), data[1]] for data in train_data]
        res = sorted(test_dist)[0:1][0][1]
        return res

    def decoding(encode_object, res):  # Декодирование результирующей таблицы
        answer = encode_object.inverse_transform([res])
        return answer

    count = sum(dataframe.isnull().sum())
    df, encod = encoding(dataframe)
    countful = count
    while count > 0:
        for column in dataframe:
            for row in range(len(dataframe)):
                if dataframe[column][row] is np.nan:
                    new = new_dataframe(column, df)
                    train_zip, test_obj = preparing_data(df, new, column, row)
                    result=nn(train_zip,test_obj)
                    result=decoding(encod,result)
                    dataframe[column][row]=result
        count = sum(dataframe.isnull().sum())
    #         print(countful-count,"из",countful)

    return dataframe


In [7]:
find_KNN_for_fillna(frame)

c:\users\anton\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\anton\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ValueError: could not convert string to float: 'BrkFace'